In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#on the first level of filtering, I want to get rid of identical networks

#on the second level of filtering, I want to get rid of networks in which
#all interchain Hbond acceptors are also satisfied by intrachain Hbonds
#I think I have to worry less about Hbond donors

In [3]:
from glob import glob
import tqdm.notebook as tqdm
from collections import defaultdict
import random

#be careful, right now I don't have any "base designs", i.e. scaffolds with different lengths that I can't compare between
base_designs = glob("/net/scratch/rdkibler/201224_rotor_hbnet/*")

In [4]:
#TEMPORARY. DO NOT USE FOR AFTER REBUILDING
base_designs = ["/net/scratch/rdkibler/201224_rotor_hbnet/"]


In [5]:
base_designs

['/net/scratch/rdkibler/201224_rotor_hbnet/']

In [6]:
get_hbnet_comments = False

class NetworkDef:
    def __init__(self,line):
        self.netid,self.residues_str,self.size,self.score,self.num_hbonds,self.percent_hbond_capacity,self.num_unsat_Hpol,self.interf_hbs,self.helics_contacted = line.strip().split()
        self.score = float(self.score)
        self.size = int(self.size)
        self.num_hbonds = int(self.num_hbonds)
        self.percent_hbond_capacity = float(self.percent_hbond_capacity)
        self.num_unsat_Hpol = int(self.num_unsat_Hpol)
        self.interf_hbs = int(self.interf_hbs)
        self.helics_contacted = int(self.helics_contacted)
    
    def __repr__(self):
        return "\t".join([self.netid,self.residues_str,str(self.size),str(self.score),str(self.num_hbonds),str(self.percent_hbond_capacity),str(self.num_unsat_Hpol),str(self.interf_hbs),str(self.helics_contacted)])
    
    def __eq__(self,other):
        return self.residues_str == other.residues_str
    
    def __ne__(self,other):
        return not self == other
    
    def __lt__(self,other):
        return float(self.score) < float(other.score)
    
    def __gt__(self,other):
        return float(self.score) > float(other.score)
    
    def __le__(self,other):
        return not self > other
    
    def __ge__(self,other):
        return not self < other
    
    def __hash__(self):
        return hash(self.residues_str)

In [7]:
pbar_outer = tqdm.tqdm(base_designs)

In [8]:
dir(pbar_outer)

['__bool__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_comparable',
 '_decr_instances',
 '_get_free_pos',
 '_instances',
 '_lock',
 '_time',
 'ascii',
 'avg_time',
 'bar_format',
 'clear',
 'close',
 'container',
 'desc',
 'disable',
 'display',
 'dynamic_miniters',
 'dynamic_ncols',
 'ema',
 'external_write_mode',
 'format_dict',
 'format_interval',
 'format_meter',
 'format_num',
 'format_sizeof',
 'fp',
 'get_lock',
 'gui',
 'iterable',
 'last_print_n',
 'last_print_t',
 'leave',
 'lock_args',
 'maxinterval',
 'mininterval',
 'miniters',
 'monitor',
 'monitor_interval',
 'moveto'

In [9]:
pbar_outer.update(1)

In [10]:
glob(base_designs[5]+"/*.pdb")

IndexError: list index out of range

In [32]:
pbar_outer = tqdm.tqdm(base_designs)
pbar_outer.position = 1
pbar_inner = tqdm.tqdm()
pbar_inner.position = 2

pass_counter = 0
all_counter = 0

# with open('save_passing_retry210223.txt','w') as f:
#     pass

for base_design in pbar_outer:
    print(base_design)
    base_design_name = base_design.split("/")[0]
    print("BASE DESIGN: ", base_design_name)
    pdbs = glob(base_design+"/*/*/*.pdb")
    designs_nets = {}

    pbar_inner.reset()
    pbar_inner.total = len(pdbs)
    pbar_inner.iterable = pdbs
    pbar_inner.refresh()
    per_design_required_interchain_hbonds = []
    per_design_network_defs = []
    
    accepted_nets_designs = defaultdict(lambda:[])

    for pdb in pbar_inner:
        max_num_resis_per_chain = 0
        network_defs = []
        hbond_set = set()
        current_hbnet = ""
        with open(pdb,'r') as f:

            res_chain_dict = {}
            get_hbnet_comments = False
            #print(pdb)
            for line in f:
                #print(line,end="")
                if line.startswith("ATOM"):
                    max_num_resis_per_chain = int(line.strip().split()[5])
                    chain = line[21]
                    resn = int(line[22:26])
                    res_chain_dict[resn] = chain

                
                #print(line,end="")
                if line == "##Begin comments##\n":
                    get_hbnet_comments = True
                if line.startswith("##End comments##"):
                    get_hbnet_comments = False

                if get_hbnet_comments:
                    #print(line)
                    if line.startswith("##network"):
                        network_defs.append(NetworkDef(line))
                        current_hbnet = line.split()[0]
                        #print(line,end="")
                        continue
                    if line.startswith("#AtomPair"):
                        sline = line.strip().split()
                        donor_atom = sline[3]
                        donor_resi = int(sline[4]) % max_num_resis_per_chain
                        acceptor_atom = sline[1]
                        acceptor_resi = int(sline[2]) % max_num_resis_per_chain
                        #print(f"hbond atom {donor_atom} at {donor_resi} to atom {acceptor_atom} at {acceptor_resi}")
                        hbond_set.add((donor_atom,donor_resi,acceptor_atom,acceptor_resi,current_hbnet))
                        continue


                if line.startswith("core_pymol_selection select core"):
                    sline = line.split()
                    ###WARNING WARNING WARNING### HARDCODED FOR TWO CHAINS ONLY ONLY ONLY
                    core_resis = set([int(resi) for resi in sline[7][:-1].split(",")])
                    core_resis = core_resis.union(set([int(resi) for resi in sline[13][:-1].split(",")]))
                    #print("core_resis",core_resis)
                    #print(line)
                if line.startswith("HBNet_NumUnsatHpol"):
                    sline = line.split()
                    num_unsat_hpol = int(sline[1])
                    #print("num_unsat_hpol",num_unsat_hpol)
                if line.startswith("HBNet_Saturation"):
                    sline = line.split()
                    hbnet_sat = sline[1]
                    #print("hbnet_sat",hbnet_sat)
                if line.startswith("HBNet_Score"):
                    sline = line.split()
                    hbnet_score = sline[1]
                    #print("hbnet_score",hbnet_score)
                if line.startswith("spanning_core_hbnets_C"):
                    sline = line.split()
                    spanning_c = sline[1]
                    #print("spanning_c",spanning_c)
                if line.startswith("spanning_core_hbnets_N"):
                    sline = line.split()
                    spanning_n = sline[1]
                    #print("spanning_n",spanning_n)


        intrachain_hbonds = []
        buried_interchain_hbonds = []
        #print(hbond_set)
        
        for donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet in hbond_set:
            #print(donor_resi,donor_atom,acceptor_resi,acceptor_atom,hbnet)
            if res_chain_dict[donor_resi] == res_chain_dict[acceptor_resi]:
                #print("intrachain")
                intrachain_hbonds.append((donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet))

                #fully buried interchain hbonds
            else:
                
                if donor_resi in core_resis and acceptor_resi in core_resis:
                    #print("buried")

                    buried_interchain_hbonds.append((donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet))

        #now I must determine if I would create a buried unsat if I mismatched these.
        #this is not the best way to check b/c I completely ignore unsatisfied donors but it's a start
        required_interchain_hbonds = []
        for interchain_donor_atom,interchain_donor_resi,interchain_acceptor_atom,interchain_acceptor_resi,interchain_hbnet in buried_interchain_hbonds:
            good = True
            for intrachain_donor_atom,intrachain_donor_resi,intrachain_acceptor_atom,intrachain_acceptor_resi,intrachain_hbnet in intrachain_hbonds:
                if interchain_acceptor_atom == intrachain_acceptor_atom and interchain_acceptor_resi == intrachain_acceptor_resi:
                    good = False
                    break
            if good:
                #print("yo!")
                required_interchain_hbonds.append((interchain_donor_atom,interchain_donor_resi,interchain_acceptor_atom,interchain_acceptor_resi,interchain_hbnet))

        per_design_required_interchain_hbonds.append(required_interchain_hbonds)
        per_design_network_defs.append(network_defs)


        #print("REQUIRED INTERCHAIN HBONDS")
        #print(pdb,required_interchain_hbonds)
        max_num_unsat_Hpol = 2
        min_percent_hbond_capacity = 0.65
        min_required_interchain_hbonds = 1

        if len(required_interchain_hbonds) >= min_required_interchain_hbonds and sum([net.num_unsat_Hpol for net in network_defs]) <= max_num_unsat_Hpol and all([net.percent_hbond_capacity >= min_percent_hbond_capacity for net in network_defs]):
            important_netids = [x[4] for x in required_interchain_hbonds]
            important_nets = tuple([net for net in network_defs if net.netid in important_netids])
            #designs_nets[pdb] = network_defs
            tiebreaker_score = sum([net.score for net in network_defs])
        #         designs_nets[pdb] = (important_nets,sum_scores_all)        


            accepted_nets_designs[important_nets].append((pdb,tiebreaker_score,important_nets))

    keep_top_n = 3

    
    #collapse the networks a bit into similar groups ignoring backbone and the difference between S and T
    grouped_accepted_nets_designs = defaultdict(lambda:[])
    for netids in accepted_nets_designs.keys():
        full_netstr = "|".join(netid.residues_str for netid in netids)
        full_netstr = full_netstr.replace("T","S").replace(",backbone","")
        grouped_accepted_nets_designs[full_netstr].extend(accepted_nets_designs[netids])
    
    
    
    passing = []
    for k in grouped_accepted_nets_designs.keys():
        passing.extend(sorted(grouped_accepted_nets_designs[k],key=lambda x:x[1])[:keep_top_n])


    with open(f"save_passing_{base_design_name}.txt",'a') as f:
        for pdb,_,_ in passing:

            f.write(pdb+"\n")
    
    pass_counter += len(passing)
    all_counter += len(pdbs)
    print(f"accepted {pass_counter} designs from {len(grouped_accepted_nets_designs)} unique network configurations out of {all_counter} total designs in the previous round")

/net/scratch/rdkibler/201224_rotor_hbnet/
BASE DESIGN:  

accepted 8576 designs from 6482 unique network configurations out of 15692 total designs in the previous round



8576

In [12]:
with open(f"save_passing_.txt","r") as f:
    print(len(f.readlines()))

19768


In [13]:
for k in sorted(accepted_nets_designs.keys(), key=lambda x:len(accepted_nets_designs[x])):
    print(len(accepted_nets_designs[k]), k[0].residues_str)

1 A_Q_28,B_S_128,B_S_144,B_T_148,B_S_167
1 A_H_7,A_W_24,B_Q_114,backbone
1 A_H_7,A_H_24,A_Q_37,B_Q_114,backbone
1 A_Q_21,B_T_125,B_N_160,A_R_18,B_S_118,B_Q_157,B_S_164,B_S_124,B_Q_165,B_T_117,backbone
1 A_R_18,A_S_21,B_E_118,B_H_151,backbone
1 A_R_18,A_Q_21,B_S_118,B_S_124,B_T_125,B_Q_157,B_N_160,B_S_164,B_Q_165,backbone
1 A_S_17,A_R_18,A_S_20,B_S_121,B_N_122,B_E_157
1 A_S_17,A_S_20,B_N_121,B_N_122,B_N_151,B_E_157
1 A_S_17,A_S_20,B_Y_109
1 A_H_17,A_S_20,B_Y_109
1 A_S_17,A_S_20,B_Y_109,B_S_118
1 A_Q_20,B_Q_109,B_S_118
1 A_N_21,A_T_22,B_S_125,B_T_152,B_T_164
1 A_H_21,B_Q_118,B_Q_125,B_T_148,B_S_157,B_Q_160,backbone
1 A_H_21,A_S_25,B_Q_118,B_Q_125,B_S_148,B_S_157,B_Q_160,backbone
1 A_H_21,B_Q_118,B_Q_125,B_S_148,B_Q_160,backbone
1 A_N_21,B_Q_118,B_S_121,B_T_125,B_N_160,B_S_163,B_Q_164,backbone
1 A_S_21,B_T_125,B_S_151
1 A_S_21,B_T_125,B_S_151,B_T_164
1 A_H_17,A_S_21,B_H_151
1 A_N_21,A_S_28,B_H_125,B_S_129,B_T_152
1 A_H_28,B_S_129,B_R_130,B_S_132,B_S_144
1 A_H_28,B_S_129,B_S_132,B_S_144
1 

In [15]:
len(accepted_nets_designs)

7676

In [14]:
counter = 0
max_count = 50

    
# for k in sorted(accepted_nets_designs.keys(), key=lambda x:len(accepted_nets_designs[x])):
#     print(len(accepted_nets_designs[k]), k[0].residues_str)

In [16]:
len(grouped_accepted_nets_designs)

6482